# Scrapping UP election data from ECI website
* This notebook contains functions which scrapes data from [link](https://results.eci.gov.in/ResultAcGenMar2022/partywiseresult-S05.htm)
* This page contains data for many states but we want only for **Uttar Pradesh.** But functions that are defined below can be used for any state.

## Importing Packages

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np


### 1. Function state_result
<img src="assets\img_3.png" alt="img" style="height: 300; width:500px;"/>
 * This function returns a dataframe containing number of seats party-wise as shown in the above image for the selected state

In [141]:

dict = {"Andhra Pradesh": "S01", "Assam": "S03", "Bihar": "S04", "Goa": "S05", "Haryana": "S07",
        "Himachal Pradesh": "S08", "Karnataka": "S10", "Madhya Pradesh": "S12", "Maharashtra": "S13", "Manipur": "S14",
        "Meghalaya": "S15", "Mizoram": "S16", "Nagaland": "S17", "Punjab": "S19", "Rajasthan": "S20",
        "Uttar Pradesh": "S24","Chhattisgarh":"S26", "Uttarakhand": "S28", "Telangana": "S29", "West Bengal": "S25",
        "Dadra & Nagar Haveli And Daman & Diu": "U04"}

def state_result(state,url):

        dict = {"Andhra Pradesh": "S01", "Assam": "S03", "Bihar": "S04", "Goa": "S05", "Haryana": "S07",
        "Himachal Pradesh": "S08", "Karnataka": "S10", "Madhya Pradesh": "S12", "Maharashtra": "S13", "Manipur": "S14",
        "Meghalaya": "S15", "Mizoram": "S16", "Nagaland": "S17", "Punjab": "S19", "Rajasthan": "S20",
        "Uttar Pradesh": "S24","Chhattisgarh":"S26", "Uttarakhand": "S28", "Telangana": "S29", "West Bengal": "S25",
        "Dadra & Nagar Haveli And Daman & Diu": "U04"}

        base_url = "https://results.eci.gov.in/"+url+dict[state]+".htm?st="+dict[state]

        response  = requests.get(base_url)

        html = response.content

        soup = BeautifulSoup(html,'lxml')

#         with open('eci_results.html', 'wb') as file:
#                 file.write(soup.prettify('utf-8'))


        r = soup.find_all('table',)

        useful = r[8]


        text = useful.text

        n = r[8].find_all('th')



        columns=[]
        for i in range(0,len(n)):
            columns.append(n[i].string)






        base_url = 'https://results.eci.gov.in/'
        response = requests.get(base_url)
        html = response.content

        soup = BeautifulSoup(html,'lxml')

        links = soup.find_all("a")

#         href=[]
#         text=[]
#         for i in range(0,len(links)-1):
#             href.append(links[i]['href'])
#             text.append(links[i].text)

        href = ['https://results.eci.gov.in/ResultAcGenMar2022/partywiseresult-S05.htm','https://results.eci.gov.in/ResultAcByeMar2022/partywiseresult-S03.htm',"https://results.eci.gov.in/ResultAcByeJune2022/partywiseresult-S01.htm"]
        text = [' GENERAL Elections to Assembly Constituency March-2022 ',' BYE Elections  to Assembly Constituency March-2022 ','BYE Elections  to Assembly Constituency June-2022' ]


        m = r[8].find_all('td')
        lst = []
        for i in m:
            lst.append(i.text.strip())
        if url==href[1][0:href[1].index("-")+1]:
            columns = columns[2:]

        if url!=href[1][0:href[1].index("-")+1]:
            l = lst[0].index("\n")


        #state = lst[0][0:l]

        heading = lst[1]

        list =[]
        if url!=href[1][0:href[1].index("-")+1]:
            n = (int)((len(lst)-2)/4)
            for i in range(0,n):
                row=lst[2+4*i:2+4*i+4]
                list.append(row)
            data = pd.DataFrame(list,columns = columns)
        else:
            n = (int)((len(lst))/4)
            for i in range(0,n):
                row=lst[4*i:4*i+4]
                list.append(row)
            data = pd.DataFrame(list,columns = columns)

        return (data)




* Below two cell contains code for **creating dataframe** for Uttar Pradesh and saving it as **UP_result.csv**

In [142]:
data = state_result('Uttar Pradesh','ResultAcGenMar2022/partywiseresult-')

In [143]:
data.to_csv('UP_result.csv',index=False)

### 2. Function votes

<img src="assets\img_4.png" alt="img" style="height: 300; width:500px;"/>

* This function returns vote percentage wise result for each party scrapping data from pie-chart for the selected state


In [148]:
#this functions returns party wise vote share
def votes(state, url):
    
    dict = {"Andhra Pradesh": "S01", "Assam": "S03", "Bihar": "S04", "Goa": "S05", "Haryana": "S07",
        "Himachal Pradesh": "S08", "Karnataka": "S10", "Madhya Pradesh": "S12", "Maharashtra": "S13", "Manipur": "S14",
        "Meghalaya": "S15", "Mizoram": "S16", "Nagaland": "S17", "Punjab": "S19", "Rajasthan": "S20",
        "Uttar Pradesh": "S24","Chhattisgarh":"S26", "Uttarakhand": "S28", "Telangana": "S29", "West Bengal": "S25",
        "Dadra & Nagar Haveli And Daman & Diu": "U04"}

    base_url = "https://results.eci.gov.in/" + url + dict[state] + ".htm?st=" + dict[state]
    response = requests.get(base_url)
    html = response.content

    soup = BeautifulSoup(html)

    vote = soup.find_all('script', type='text/javascript')

    text = vote[2].string
    text = text.split(';')
    text = text[8].strip()
    text = text.replace('[', ' ')
    text = text.replace('(', '')
    text = text.replace('{', ' ')
    text = text.replace('}', ' ')
    text = text.replace(']', ' ')
    text = text.replace(')', '')
    text = text.replace("'", ' ')
    text = text.replace(",", ' ')
    text = text.replace('%',"")
    text = text.split()
    text = text[1:]

    lst = []
    for i in range(0, len(text), 3):
        lst.append(text[i:i + 3])

    data = pd.DataFrame(lst, columns=["Party", 'Vote Percentage', 'Votes'])
    data[['Vote Percentage', 'Votes']] = data[['Vote Percentage', 'Votes']].apply(pd.to_numeric)
    data.index+=1
    return (data)

In [149]:
data=votes('Uttar Pradesh','ResultAcGenMar2022/partywiseresult-')
data.to_csv('up_votepercentage.csv',index=False)

### 3. Funtion state_names
 <img src="assets\img_5.png" alt="img" style="height: 300; width:300px;"/>
 * This function returns list of state results present in one link of ECI websites

In [146]:
#this function returns list of states in selected link
def state_names(url):
    base_url = "https://results.eci.gov.in/"+url
    response = requests.get(base_url)

    html = response.content

    soup = BeautifulSoup(html,'lxml')


    select = soup.find_all('select')



    s = select[0].find_all('option')


    state=[]
    for i in range(0,len(s)):
        state.append(s[i].text)

    return(state)



In [147]:
state_names('ResultAcGenMar2022/partywiseresult-S05.htm')


['Goa', 'Manipur', 'Punjab', 'Uttar Pradesh', 'Uttarakhand']

### 4. Function constituencies_dict
 <img src="assets\img_6.png" alt="img" style="height: 500; width:300px;"/>
* This function returns a dictionary with constituency names as keys and and constituency number as values form the specified state 

In [106]:
#This function returns dict of constituency and their number
def constituencies_dict(url,state):
    
    dict = {"Andhra Pradesh": "S01", "Assam": "S03", "Bihar": "S04", "Goa": "S05", "Haryana": "S07",
        "Himachal Pradesh": "S08", "Karnataka": "S10", "Madhya Pradesh": "S12", "Maharashtra": "S13", "Manipur": "S14",
        "Meghalaya": "S15", "Mizoram": "S16", "Nagaland": "S17", "Punjab": "S19", "Rajasthan": "S20",
        "Uttar Pradesh": "S24","Chhattisgarh":"S26", "Uttarakhand": "S28", "Telangana": "S29", "West Bengal": "S25",
        "Dadra & Nagar Haveli And Daman & Diu": "U04"}

    base_url='https://results.eci.gov.in/'+url[0:(url.index('/')+1)]+'Constituencywise'+dict[state]+'1.htm?ac=1'
    response = requests.get(base_url)
    html = response.content

    soup = BeautifulSoup(html,'lxml')


    input_id = soup.find_all('input',id=dict[state])


    txt= str(input_id[0])


    start=txt.index('value=')+7

    txt=txt[start:-3]


    txt = txt.replace(' ',"")

    txt=txt.replace(',',' ')



    txt= txt.replace(';',' ')
    lst=txt.split()


    dict_cons={}
    for i in range(0,len(lst),2):
        dict_cons[lst[i+1]]=lst[i]
    return(dict_cons)


In [107]:
dict = constituencies_dict("ResultAcGenMar2022/partywiseresult-S05.htm",'Uttar Pradesh')
data = pd.DataFrame({'Constituency':list(dict.keys()),'Constituency number':list(dict.values())})
data.to_csv('UP_Constituency.csv',index=False)

### 5. Function candidate_wise_result
 <img src="assets\img_7.png" alt="img" style="height: 1000; width:500px;"/>
 * This function returns dataframe containing result in above image format for the state and constituency specified

In [122]:
#this function returns candidate wise result
def candidate_wise_result(url,state,constituency):
    dict = {"Andhra Pradesh": "S01", "Assam": "S03", "Bihar": "S04", "Goa": "S05", "Haryana": "S07",
        "Himachal Pradesh": "S08", "Karnataka": "S10", "Madhya Pradesh": "S12", "Maharashtra": "S13", "Manipur": "S14",
        "Meghalaya": "S15", "Mizoram": "S16", "Nagaland": "S17", "Punjab": "S19", "Rajasthan": "S20",
        "Uttar Pradesh": "S24","Chhattisgarh":"S26", "Uttarakhand": "S28", "Telangana": "S29", "West Bengal": "S25",
        "Dadra & Nagar Haveli And Daman & Diu": "U04"}
    
    base_url = 'https://results.eci.gov.in/'+url[0:(url.index('/')+1)]+'Constituencywise'+dict[state]+""+constituency+'.htm?ac='+constituency
    response = requests.get(base_url)
    html= response.content
    soup = BeautifulSoup(html,'lxml')
    table = soup.find_all('table')
    table[0]

    th = table[0].find_all('th')
    heading=[]
    for i in range(0,len(th)):
        heading.append(th[i].text)

    table[0]

    tr = table[0].find_all('tr',style="font-size:12px;")
    tr

    len(tr)

    rows=[]
    for i in range(0,len(tr)):
        td = tr[i].find_all('td')
        for j in range(0,len(td)):
            rows.append(td[j].text)


    d={}
    j=1
    for i in range(0,len(rows),7):
        d[j]=rows[i:i+8]
        j+=1
    heading.append('hello')
    data=pd.DataFrame.from_dict(d,orient='index',columns=heading)
    data.drop(['O.S.N.','hello'],axis=1,inplace=True)
    return(data)



### 6. Created data frame to store candidate wise result for each contituency

In [150]:

data = pd.DataFrame(columns=['Candidate','Party','EVM Votes','Postal Votes','Total Votes','% of Votes','Constituency'])
dict = constituencies_dict("ResultAcGenMar2022/partywiseresult-S05.htm",'Uttar Pradesh')
for i in dict:
    d=candidate_wise_result("ResultAcGenMar2022/partywiseresult-S05.htm",'Uttar Pradesh',dict[i])
    d['Constituency'] = i
    d['Constituency number'] = dict[i]
    print(d)
    data = pd.concat([data,d],sort=False,ignore_index=True)

data

                     Candidate                          Party EVM Votes  \
1                Kunwar  Chand                Samajwadi Party     68803   
2             Dr. G S Dharmesh         Bharatiya Janata Party    117352   
3   Dr.  Bhartendra Kumar Arun            Bahujan Samaj Party     54216   
4               Sikandar Singh       Indian National Congress      5548   
5                   Prem Singh                Aam Aadmi Party      1114   
6                         Mona                    Peace Party       419   
7                 Sundar Singh  Bharatiya Majdoor Janta Party       420   
8                   Akash Soni                    Independent       363   
9              Prakash Indiver                    Independent       617   
10                 Hari Kishan                    Independent       552   
11                        NOTA              None of the Above      1414   

   Postal Votes Total Votes % of Votes Constituency Constituency number  
1           296       690

                   Candidate                                Party EVM Votes  \
1         Ambrish Singh Gaur             Indian National Congress      1636   
2            Pratibha Shukla               Bharatiya Janata Party     92550   
3   Dr. Ram Prakash Kushwaha                      Samajwadi Party     78996   
4            Vinod Kumar Pal                  Bahujan Samaj Party     32136   
5              Kamlesh Kumar                    Jan Adhikar Party       615   
6                Nausad Babu                          Peace Party       230   
7               Rajesh Kumar  Peoples Party of India (Democratic)       622   
8               Vivesh Kumar                      Aam Aadmi Party      1129   
9               Brij Bhushan                          Independent       432   
10                 Rameshwar                          Independent       676   
11               Vipin Kumar                          Independent       808   
12                      NOTA                    None

                Candidate                                Party EVM Votes  \
1             RICHA SINGH                      Samajwadi Party     88372   
2             GULAM QADIR                  Bahujan Samaj Party      7575   
3           TASLEEM UDDIN             Indian National Congress      2288   
4     SIDHARTH NATH SINGH               Bharatiya Janata Party    118383   
5   AJEET KUMAR CHAUDHARY  Peoples Party of India (Democratic)      1085   
6     KAMLESH KUMAR SINGH               Parivartan Samaj Party       360   
7           DILSHAD AHMAD    National Democratic Peoples Front       260   
8    ADV. SUSHMITA RAGHAV                      Aam Aadmi Party       595   
9        AJAY KUMAR GUPTA                          Independent       459   
10            KAMAR JAHAN                          Independent       358   
11     GANESH JI TRIPATHi                          Independent       337   
12        SAKLAIN MUSHTAK                          Independent       374   
13          

              Candidate                             Party EVM Votes  \
1           MEHBOOB ALI                   Samajwadi Party    127865   
2   MOHAMMED NAVED AYAZ               Bahujan Samaj Party     34405   
3             RAM SINGH            Bharatiya Janata Party     57289   
4           SALEEM KHAN          Indian National Congress       830   
5            MOH. AHSAN  Pachchasi Parivartan Samaj Party       202   
6                 ISLAM                   Aam Aadmi Party       407   
7             SHIVKUMAR    Aazad Samaj Party (Kanshi Ram)       226   
8             AKBAR ALI                       Independent       237   
9               SHAKINA                       Independent       236   
10                 NOTA                 None of the Above       968   

   Postal Votes Total Votes % of Votes Constituency Constituency number  
1           870      128735      57.43       Amroha                  41  
2           180       34585      15.43       Amroha                  4

             Candidate                                 Party EVM Votes  \
1         RAMESH CHAND              Indian National Congress      2196   
2          SAROJ KUMAR                   Bahujan Samaj Party     50937   
3          DR. SANGRAM                       Samajwadi Party     90393   
4   UMESH KUMAR MAURYA                     Jan Adhikar Party      2176   
5      PRASHANT  SINGH  Nirbal Indian Shoshit Hamara Aam Dal     73872   
6         RAMESH KUMAR                       Aam Aadmi Party      1220   
7             RAMDHARI             Rashtriya Bhagidari Party      1910   
8       SAURABH NISHAD               Vikassheel Insaan Party      3683   
9             RAM DEVI                           Independent       812   
10     DR.SEEMA PANDEY                           Independent       849   
11                NOTA                     None of the Above      1395   

   Postal Votes Total Votes % of Votes Constituency Constituency number  
1            16        2212       0.9

...

             Candidate                           Party EVM Votes Postal Votes  \
1    AJAY PRATAP SINGH          Bharatiya Janata Party    113999          708   
2            NAVAB JAN                 Samajwadi Party    133784          607   
3          MUJAHID ALI             Bahujan Samaj Party     22041          122   
4           SALMA AAGA        Indian National Congress      1499            9   
5           JALALUDDIN  Aazad Samaj Party (Kanshi Ram)       353            2   
6   SHAH FAISAL ANSARI                 Aam Aadmi Party       317            2   
7          SURAJ SINGH           Rashtriya Samanta Dal       166            0   
8                 ABID                     Independent       154            1   
9           MOHD JAVED                     Independent       363            0   
10           SHAHNAWAJ                     Independent       420            0   
11                NOTA               None of the Above      1054            7   

   Total Votes % of Votes C

                  Candidate                         Party EVM Votes  \
1                 ADISHAKTI      Indian National Congress      2579   
2              JAYRAM SINGH           Bahujan Samaj Party     39267   
3   BRAJESH KUMAR PRAJAPATI               Samajwadi Party     57916   
4            RAMKESH NISHAD        Bharatiya Janata Party     86600   
5                RAMCHANDRA      Communist Party of India      2867   
6               MOOLCHANDRA  Bhartiya Shakti Chetna Party       912   
7              ARVIND KUMAR                   Independent      1035   
8                     UTTAM                   Independent       821   
9                  MADHURAJ                   Independent      1082   
10                     NOTA             None of the Above      2264   

   Postal Votes Total Votes % of Votes Constituency Constituency number  
1             7        2586       1.32     Tindwari                 232  
2           183       39450       20.1     Tindwari                 23

                                 Candidate  \
1   DHIRENDRA PRATAP SINGH URF DHIRU SINGH   
2                               RAM PRATAP   
3     RAMPRATAP VERMA URF SHASHIKANT VERMA   
4                               HASIB KHAN   
5                    AJAY KUMAR CHATURVEDI   
6                             ABDUL MANNAN   
7                          MUSTAQEEM AHMAD   
8                                  PREMLAL   
9                        SAHBAJ FARAZ KHAN   
10                                 SANCHIT   
11                            GYAN CHANDRA   
12                  GYANENDRA KUMAR MISHRA   
13                                    NOTA   

                                     Party EVM Votes Postal Votes Total Votes  \
1                 Indian National Congress     12907           37       12944   
2                      Bahujan Samaj Party      9623           46        9669   
3                   Bharatiya Janata Party     87027          135       87162   
4                          Sama

                 Candidate                                      Party  \
1               KALICHARAN                     Bharatiya Janata Party   
2   SAIYYADA SHADAB FATIMA                        Bahujan Samaj Party   
3             GYAN PRAKASH                   Indian National Congress   
4                   ARVIND                   Bhartiya Jan Nayak Party   
5             ARVIND VARMA                       Janata Dal  (United)   
6       OM PRAKASH RAJBHAR             Suheldev Bharatiya Samaj Party   
7                  JAI RAM                     All India Forward Bloc   
8     MAHESH SINGH CHAUHAN                  Rashtriya Lok Janta Party   
9               RAMPRAVESH  Marxist Communist Party of India (United)   
10                LALMOHAR                           Sarv Rajya Party   
11               SHIVPUJAN                            Aam Aadmi Party   
12             SHAUKAT ALI     All India Majlis-E-Ittehadul Muslimeen   
13        SATYAA DEV SINGH           Rashtrawadi Ja

,Candidate,Party,EVM Votes,Postal Votes,Total Votes,% of Votes,Constituency,Constituency number
0,Kunwar Chand,Samajwadi Party,68803,296,69099,27.44,AgraCantt.,87
1,Dr. G S Dharmesh,Bharatiya Janata Party,117352,444,117796,46.78,AgraCantt.,87
2,Dr. Bhartendra Kumar Arun,Bahujan Samaj Party,54216,193,54409,21.61,AgraCantt.,87
3,Sikandar Singh,Indian National Congress,5548,42,5590,2.22,AgraCantt.,87
4,Prem Singh,Aam Aadmi Party,1114,11,1125,0.45,AgraCantt.,87
...,...,...,...,...,...,...,...,...
4840,JANG BAHADUR,Independent,460,1,461,0.2,Zamania,379
4841,RAJENDRA PRASAD,Independent,528,2,530,0.23,Zamania,379
4842,RAMASHRAY,Independent,713,3,716,0.31,Zamania,379
4843,SARVDEO SINGH ALIAS DR. SARVDEO SINGH YOGACHAR...,Independent,536,1,537,0.23,Zamania,379


In [151]:
data

,Candidate,Party,EVM Votes,Postal Votes,Total Votes,% of Votes,Constituency,Constituency number
0,Kunwar Chand,Samajwadi Party,68803,296,69099,27.44,AgraCantt.,87
1,Dr. G S Dharmesh,Bharatiya Janata Party,117352,444,117796,46.78,AgraCantt.,87
2,Dr. Bhartendra Kumar Arun,Bahujan Samaj Party,54216,193,54409,21.61,AgraCantt.,87
3,Sikandar Singh,Indian National Congress,5548,42,5590,2.22,AgraCantt.,87
4,Prem Singh,Aam Aadmi Party,1114,11,1125,0.45,AgraCantt.,87
...,...,...,...,...,...,...,...,...
4840,JANG BAHADUR,Independent,460,1,461,0.2,Zamania,379
4841,RAJENDRA PRASAD,Independent,528,2,530,0.23,Zamania,379
4842,RAMASHRAY,Independent,713,3,716,0.31,Zamania,379
4843,SARVDEO SINGH ALIAS DR. SARVDEO SINGH YOGACHAR...,Independent,536,1,537,0.23,Zamania,379


In [152]:
data.to_csv("Candidate_wise_result.csv",index=False)